## Budget Calculator App API

This notebook serves as a model for the app API

In [186]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from io import StringIO

## Initial Requests 

This code runs each time the application starts. 

In [187]:
base_url = 'https://www.numbeo.com/cost-of-living/historical-data-city-selector'

In [188]:
page = requests.get(base_url)
numbeo_city_soup = BeautifulSoup(page.content, "html.parser")
results = numbeo_city_soup.find('table', class_='related_links')

[<tr><td style="width: 20%">
<h4>'</h4>
<a href="https://www.numbeo.com/cost-of-living/city-history/in/%27s-Hertogenbosch">'s-Hertogenbosch, Netherlands</a><br/>
<h4>A</h4>
<a href="https://www.numbeo.com/cost-of-living/city-history/in/Aachen">Aachen, Germany</a><br/>
<a href="https://www.numbeo.com/cost-of-living/city-history/in/Aalborg">Aalborg, Denmark</a><br/>
<a href="https://www.numbeo.com/cost-of-living/city-history/in/Aarhus-Denmark">Aarhus, Denmark</a><br/>
<a href="https://www.numbeo.com/cost-of-living/city-history/in/Abbotsford">Abbotsford, Canada</a><br/>
<a href="https://www.numbeo.com/cost-of-living/city-history/in/Aberdeen">Aberdeen, United Kingdom</a><br/>
<a href="https://www.numbeo.com/cost-of-living/city-history/in/Abidjan">Abidjan, Ivory Coast</a><br/>
<a href="https://www.numbeo.com/cost-of-living/city-history/in/Abu-Dhabi">Abu Dhabi, United Arab Emirates</a><br/>
<a href="https://www.numbeo.com/cost-of-living/city-history/in/Abuja">Abuja, Nigeria</a><br/>
<a href=

In [189]:
list_cities = results.find_all('a')
list_cities[0]

<a href="https://www.numbeo.com/cost-of-living/city-history/in/%27s-Hertogenbosch">'s-Hertogenbosch, Netherlands</a>

In [190]:
city_name = lambda x: f"({x[0]}) {x[1]}" if len(x) > 2 else x[0]

city_dict = lambda x: {'City':city_name(x.text.split(",")), 'Country':x.text.split(",")[-1].strip(), 'Url':x["href"]}
city_pages = [city_dict(city) for city in list_cities]

In [191]:
df = pd.DataFrame(city_pages) # creates a dataframe with all cities, their country and page urls for their data tables 
df.head()

,City,Country,Url
0,'s-Hertogenbosch,Netherlands,https://www.numbeo.com/cost-of-living/city-his...
1,Aachen,Germany,https://www.numbeo.com/cost-of-living/city-his...
2,Aalborg,Denmark,https://www.numbeo.com/cost-of-living/city-his...
3,Aarhus,Denmark,https://www.numbeo.com/cost-of-living/city-his...
4,Abbotsford,Canada,https://www.numbeo.com/cost-of-living/city-his...


## User Interaction

This part of the code kicks in when the user inputs search criterion via the UI. Country and city are case sensitive. Appropriate measures should be taken in the app logic to prevent link brakage due to incorrect user entries.

In [192]:
country_selection = input("Select Country")
country_slice = df[df["Country"] == country_selection]

In [193]:
country_slice

,City,Country,Url
452,Kabul,Afghanistan,https://www.numbeo.com/cost-of-living/city-his...


Once the data has been sliced by country, the user can select a city. Some cities are duplicated accross countries or states. Being able to avoid ambiguity by defining country then city is an essential step in ensuring the app provides accurate information.

In [194]:
def get_tables(city):
    page = requests.get(df.iloc[df.loc[df['City'] == city].index[0]]["Url"])
    one_city_soup = BeautifulSoup(page.content, "html.parser")
    inner_width = one_city_soup.find_all('div', class_='innerWidth')
    results = inner_width[2].find_all('table')
    return results

In [195]:
city = input("Enter City")
data = get_tables(city)

In [196]:
len(data)


14

A total of 14 tables are returned. 



The below function `converts the raw html tables into dataframes`. 
The dataframes are then combined into categories of cost groups which the user will be able to track their expenditure in.

The categories that will be tracked on our application include: 

- The cost of restuarant outings
- The cost of food and grocery items 
- Monthly rental expenses 
- Monthly transport expenses
- Monthly cost of utilities
- Montly expenditure on sports and leisure
- Monthly expenditure on clothing

In [197]:
def categorize_data(tables):
    """Categorize and process data from a list of tables."""
    # Convert each table in the list to DataFrame
    reader_converter = lambda x: pd.DataFrame(pd.read_html(StringIO(str(x)))[0])
    df_list = [reader_converter(table) for table in tables]

    # Categorize different types of data
    restaurants = pd.concat([df_list[0], df_list[1]], axis=1).T.drop_duplicates().T
    markets = pd.concat([df_list[2], df_list[3], df_list[4]], axis=1).T.drop_duplicates().T
    transport = pd.concat([df_list[8], df_list[9]], axis=1).T.drop_duplicates().T
    rent_per_month = df_list[5]
    utilities = df_list[11]
    leisure = df_list[12]
    clothing = df_list[13]

    # Collect categorized frames into a list
    categorized_frames = [restaurants, markets, leisure, rent_per_month, transport, utilities, clothing]

    return categorized_frames

In [198]:
categorized_data = categorize_data(data)

In [199]:
categorized_data[1]

,Year,"Milk (regular), (1 liter)",Loaf of Fresh White Bread (500g),Eggs (regular) (12),Water (1.5 liter bottle),Domestic Beer (0.5 liter bottle),Apples (1kg),Oranges (1kg),Potato (1kg),"Rice (white), (1kg)",Tomato (1kg),Banana (1kg),Onion (1kg),Local Cheese (1kg),Cigarettes 20 Pack (Marlboro),Chicken Fillets (1kg),Beef Round (1kg) (or Equivalent Back Leg Red Meat)
0,2023,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
1,2022,53.00,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
2,2021,51.11,30.00,106.71,27.50,-,68.75,96.67,28.00,97.14,34.29,87.50,22.50,325.00,148.33,218.33,470.00
3,2020,52.50,-,98.40,-,-,-,-,-,-,-,-,-,300.00,-,-,427.50
4,2019,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
5,2018,62.50,32.02,-,-,-,-,-,-,-,-,-,-,-,-,-,-


The below function cleans the categorized data, removing nulls and changing datatypes to floats. This is essential for the next step: interpolation. Interpolation fills in null values according to a prescribed pattern ("linear" in this case) ensuring continuity and userbility in the data.  

In [200]:
import pandas as pd

def clean_data(frames):
    new_frames = []
    for frame in frames:
        # Step 1: Identify columns to interpolate excluding 'Year'
        interpolate_columns = frame.columns[frame.columns != 'Year']

        # Step 2: Extract the subset of data to interpolate
        data_to_interpolate = frame[interpolate_columns].replace(to_replace="-", value=np.nan)

        # Step 3: Perform interpolation
        interpolated_data = data_to_interpolate.astype(float).interpolate(method='linear')

        # Step 4: Replace the original columns with interpolated data
        frame[interpolate_columns] = interpolated_data

        # Step 5: Append the modified frame to the list of new frames
        new_frames.append(frame)
        
    return new_frames


In [201]:
clean_data = clean_data(categorized_data)

Once interpolated, the data can be ploted as a time series (line plot showing values from 2011 to 2023 for each feature).

In [202]:
titles = ["restaurants", "markets", "leisure", "rent", "transport", "utilities", "clothing"]
records = {title: df.to_dict(orient='records') for title, df in zip(titles, clean_data)}

In [203]:
records["restaurants"][0]

{'Year': 2023,
 'Meal, Inexpensive Restaurant': 157.5,
 'Meal for 2 People, Mid-range Restaurant, Three-course': nan,
 'McMeal at McDonalds (or Equivalent Combo Meal)': nan,
 'Domestic Beer (0.5 liter draught)': nan,
 'Coke/Pepsi (0.33 liter bottle)': 21.25,
 'Water (0.33 liter bottle)': 10.0,
 'Cappuccino (regular)': nan}

In [204]:
records["markets"][0]

{'Year': 2023,
 'Milk (regular), (1 liter)': nan,
 'Loaf of Fresh White Bread (500g)': nan,
 'Eggs (regular) (12)': nan,
 'Water (1.5 liter bottle)': nan,
 'Domestic Beer (0.5 liter bottle)': nan,
 'Apples (1kg)': nan,
 'Oranges (1kg)': nan,
 'Potato (1kg)': nan,
 'Rice (white), (1kg)': nan,
 'Tomato (1kg)': nan,
 'Banana (1kg)': nan,
 'Onion (1kg)': nan,
 'Local Cheese (1kg)': nan,
 'Cigarettes 20 Pack (Marlboro)': nan,
 'Chicken Fillets (1kg)': nan,
 'Beef Round (1kg) (or Equivalent Back Leg Red Meat)': nan}

In [205]:
records["leisure"][0]

{'Year': 2023,
 'Fitness Club, Monthly Fee for 1 Adult': nan,
 'Tennis Court Rent (1 Hour on Weekend)': nan,
 'Cinema, International Release, 1 Seat': nan}

In [206]:
records["rent"][0]

{'Year': 2023,
 'Apartment (1 bedroom) in City Centre': nan,
 'Apartment (1 bedroom) Outside of Centre': nan,
 'Apartment (3 bedrooms) in City Centre': 17500.0,
 'Apartment (3 bedrooms) Outside of Centre': nan}

In [207]:
records["transport"][0]

{'Year': 2023,
 'One-way Ticket (Local Transport)': nan,
 'Gasoline (1 liter)': nan,
 'Monthly Pass (Regular Price)': nan}

In [208]:
records["utilities"][0]

{'Year': 2023,
 'Basic (Electricity, Heating, Cooling, Water, Garbage) for 85m2 Apartment': nan,
 'Internet (60 Mbps or More, Unlimited Data, Cable/ADSL)': nan,
 'Mobile Phone Monthly Plan with Calls and 10GB+ Data': nan}

In [209]:
records["clothing"][0]

{'Year': 2023,
 '1 Pair of Jeans (Levis 501 Or Similar)': nan,
 '1 Summer Dress in a Chain Store (Zara, H&M, ...)': nan,
 '1 Pair of Nike Running Shoes (Mid-Range)': nan,
 '1 Pair of Men Leather Business Shoes': nan}

In [210]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

## Helper Functions

#### Restaurants Estimate

In [ ]:
restaurants = clean_data[0]
est_restaurant = round(restaurants.iloc[0:1,1:5].sum().sum(), 2)
est_restaurant

977.64

#### Market Estimate

In [ ]:
market = clean_data[1]
est_market = round(market.iloc[0:1,1:].sum().sum() * 2, 2)
est_market

1660.8

#### Leisure Average

In [ ]:
leisure = clean_data[2]
est_leisure = round(leisure.iloc[0:1,1:].sum().sum(), 2)
est_leisure

793.5

#### Rental Average

In [ ]:
rental = clean_data[3]
est_rent = round((rental.iloc[0:1,1:3].sum().mean() + rental.iloc[0:1,3:].sum().mean() / 3) / 2, 2)
est_rent

8320.23

#### Public Transport 

In [ ]:
transport = clean_data[4]
est_transport = round((
    transport.iloc[0:1,3:].sum().sum() + 
    transport.iloc[0:1,2:3].sum().sum() * 30 + 
    transport.iloc[0:1,1:2].sum().sum() * 44 ) / 3, 2)
est_transport 

746.54

#### Utilites 

In [ ]:
utilities = clean_data[5]
est_utilities = round(utilities.iloc[0:1,1:].sum().sum() / 2, 2)
est_utilities 

1514.33

#### Clothing

In [ ]:
clothing = clean_data[6]

est_clothing = round((clothing.iloc[0:1:,1:2].sum().sum() * 4 +
    clothing.iloc[0:1,2:3].sum().sum() * 12 +
    clothing.iloc[0:1,3:4].sum().sum() * 2 +
    clothing.iloc[0:1,4:].sum().sum() * 2
) / 12, 2)

est_clothing

1301.13

In [ ]:
total = sum([est_rent, est_restaurant, est_market, est_leisure, est_transport, est_clothing, est_utilities])
round(total, 2)

15314.17

In [ ]:
break

In [ ]:
import os

def save_df(df_list, city): 
    categories=["Market","Leisure","Rental","Transport", "Utilities","Clothing"]
    # Create directory if it doesn't exist
    directory = f'data/processed/{city}'
    if not os.path.exists(directory):
        os.makedirs(directory)
    for index, frame in enumerate(df_list):
        # Ensure the title index is within bounds
        frame.astype(float).interpolate()
        if index < len(categories):
            # Save DataFrame to CSV
            frame.to_csv(f'{directory}/{categories[index]}.csv', sep=',', index=False, encoding='utf-8')

In [ ]:
save_df(clean_data, 'nairobi')
